In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import log_loss
import optuna
import gc
xgb.__version__

'1.6.2'

In [3]:
!nvidia-smi

Sat Dec 24 10:13:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   35C    P8    32W / 280W |    905MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0  On |                  N/A |
| 41%   

In [4]:
!nvidia-smi -L

GPU 0: NVIDIA TITAN RTX (UUID: GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f)
GPU 1: NVIDIA TITAN RTX (UUID: GPU-50aeb092-88f5-4e0b-7f73-32741666f319)


In [5]:
train_x = dd.read_csv('../input/otto_ii/xgtrain_25_3XNaN.csv')
test_x = dd.read_csv('../input/otto_ii/xgval.csv')

In [6]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [7]:
cluster = LocalCUDACluster(n_workers=2)
client = Client(cluster)

2022-12-24 10:13:48,586 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-uqpfsgux', purging
2022-12-24 10:13:48,586 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-fa4z4ku4', purging
2022-12-24 10:13:48,587 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-24 10:13:48,587 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-12-24 10:13:48,609 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-24 10:13:48,609 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [8]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [9]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)


In [10]:
num_round = 1000

In [11]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['multi:softprob']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 2, 25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),
        'num_class':9,

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    lloss = log_loss(test_y, predictions)
    
    return lloss

In [12]:
study = optuna.create_study(direction='minimize')

[I 2022-12-24 10:13:57,071] A new study created in memory with name: no-name-111a6bce-bbd4-4231-903a-a0cd9400c7e8


In [13]:
%%time
study.optimize(objective, n_trials=2)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[10:13:57] task [xgboost.dask-0]:tcp://127.0.0.1:45925 got new rank 0
[10:13:57] task [xgboost.dask-1]:tcp://127.0.0.1:40165 got new rank 1
[I 2022-12-24 10:15:36,278] Trial 0 finished with value: 0.7373675216317068 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 0.43938733791592527, 'alpha': 0.050604229839500206, 'colsample_bytree': 0.6398651625741203, 'subsample': 0.7414084611905636, 'learning_rate': 0.0031662307224589186, 'max_depth': 7, 'min_child_weight': 281, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.7373675216317068.
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[10:15:36] task [xgboost.dask-0]:tcp://127.0.0.1:45925 got new r

CPU times: user 23.2 s, sys: 21.9 s, total: 45.1 s
Wall time: 18min 23s


In [14]:
%%time
study.optimize(objective, n_trials=100)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[10:32:21] task [xgboost.dask-0]:tcp://127.0.0.1:45925 got new rank 0
[10:32:21] task [xgboost.dask-1]:tcp://127.0.0.1:40165 got new rank 1
[I 2022-12-24 10:33:45,923] Trial 2 finished with value: 0.4672324982604495 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 0.0019170784354076047, 'alpha': 0.1291234565979344, 'colsample_bytree': 0.6607590830300893, 'subsample': 0.6853477991275184, 'learning_rate': 0.04559226315600884, 'max_depth': 8, 'min_child_weight': 113, 'eval_metric': 'logloss'}. Best is trial 2 with value: 0.4672324982604495.
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[10:33:45] task [xgboost.dask-0]:tcp://127.0.0.1:45925 got new ran

CPU times: user 18min 48s, sys: 10min 14s, total: 29min 2s
Wall time: 12h 18min 3s


In [15]:
study.best_trial.params

{'objective': 'multi:softprob',
 'tree_method': 'gpu_hist',
 'lambda': 0.002394506532951713,
 'alpha': 0.04348807407534056,
 'colsample_bytree': 0.3981624329956984,
 'subsample': 0.9499449058081066,
 'learning_rate': 0.023004997284942125,
 'max_depth': 16,
 'min_child_weight': 131,
 'eval_metric': 'logloss'}

In [16]:
real_test_x = dd.read_csv('../input/otto_ii/xgtest.csv')

In [17]:
real_test_y = real_test_x['target']
real_test_x = real_test_x[real_test_x.columns.difference(['target'])]

In [18]:
drealtest = xgb.dask.DaskDMatrix(client, real_test_x, real_test_y)

In [19]:
params = study.best_trial.params
params['num_class']= 9

output = xgb.dask.train(client, params, dtrain, num_round)
    
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})


predictions = xgb.dask.predict(client, booster, drealtest)
    
predictions = predictions.compute()

log_loss(real_test_y, predictions)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[22:50:25] task [xgboost.dask-0]:tcp://127.0.0.1:45925 got new rank 0
[22:50:25] task [xgboost.dask-1]:tcp://127.0.0.1:40165 got new rank 1


0.4421982133597365

In [20]:
%%time
params = study.best_trial.params
params['seed'] = 2022
params['num_class']= 9

output = xgb.dask.train(client, params, dtrain, num_round)
    
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})

predictions = xgb.dask.predict(client, booster, drealtest)
    
predictions = predictions.compute()

log_loss(real_test_y, predictions)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[22:54:06] task [xgboost.dask-0]:tcp://127.0.0.1:45925 got new rank 0
[22:54:06] task [xgboost.dask-1]:tcp://127.0.0.1:40165 got new rank 1


CPU times: user 6.39 s, sys: 3.17 s, total: 9.56 s
Wall time: 3min 42s


0.4428701726909696

In [21]:
%%time 
scores = []

for jj in range(15):
    print(jj)
    params = study.best_trial.params
    params['num_class']= 9
    params['seed'] = 5*jj**3 + 137

    output = xgb.dask.train(client, params, dtrain, num_round)

    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, drealtest)

    predictions = predictions.compute()
    
    error = log_loss(real_test_y, predictions)

    scores.append(error)
    print(error)

0


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[22:57:48] task [xgboost.dask-0]:tcp://127.0.0.1:45925 got new rank 0
[22:57:48] task [xgboost.dask-1]:tcp://127.0.0.1:40165 got new rank 1


0.4421940714806057
1


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[23:01:31] task [xgboost.dask-0]:tcp://127.0.0.1:45925 got new rank 0
[23:01:31] task [xgboost.dask-1]:tcp://127.0.0.1:40165 got new rank 1


0.442816466264247
2


...

0.44239127479245605
13


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[23:45:59] task [xgboost.dask-0]:tcp://127.0.0.1:45925 got new rank 0
[23:45:59] task [xgboost.dask-1]:tcp://127.0.0.1:40165 got new rank 1


0.4420125846424357
14


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[23:49:42] task [xgboost.dask-0]:tcp://127.0.0.1:45925 got new rank 0
[23:49:42] task [xgboost.dask-1]:tcp://127.0.0.1:40165 got new rank 1


0.4428726233586265
CPU times: user 1min 31s, sys: 52.4 s, total: 2min 23s
Wall time: 55min 35s


In [22]:
scores

[0.4421940714806057,
 0.442816466264247,
 0.44268883831902606,
 0.44190821703112654,
 0.44241134401158183,
 0.4423410724311385,
 0.44147628826952695,
 0.44242275319630847,
 0.4427715475779431,
 0.4419249379504423,
 0.44190910936172084,
 0.4411187546074096,
 0.44239127479245605,
 0.4420125846424357,
 0.4428726233586265]

In [23]:
import numpy as np
np.mean(scores)

0.442217325552973

/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/